- notebook will be part of a series
- long term goal will be a bilevel programm to optimize bed planning in the private garden
- current idea:
    - upper level problem: "knappsack problem"
    - lower level problem: "strip packing problem"

## outline

1. introduce knappsack problem
1. math description
1. pyomo implementation
1. example

# algebraic formulation - 0-1 knappsack problem

$$
\begin{array}{lll}
\max & \sum_i v_i x_i & \\
s.t. & \sum_i w_i x_i \leq W & \\
     & x_i \in\{0,1\}
\end{array}
$$

## sets

- $I$, set of items $i$

## (decision) variables

- $x_i$, binary, 1 iff item $i$ is put into knappsack

## parameter

- $v_i$ value of item $i$
- $w_i$ weight of item $i$
- $W$ capacity of knappsack


In [ ]:
import pyomo.environ as pyo
import pandas as pd
import numpy as np

In [ ]:
# data
items = ['a','b','c','d']
values = [8,3,6,11]
weights= [5,7,4,3]
capacity_limit = 14

df = pd.DataFrame(
    {'items' : items,
     'values' : values,
     'weights' : weights},
    )
df['capacity'] = capacity_limit

display(df)

data = {
    'model_name': 'bounded_knappsack',
    'problem_data' : df,
    'solver': 'cbc'
}

,items,values,weights,capacity
0,a,8,5,14
1,b,3,7,14
2,c,6,4,14
3,d,11,3,14


In [ ]:
# model

def knappsack_01(data):
    pdata = data['problem_data']
    
    m = pyo.ConcreteModel(data['model_name'])
    
    ## sets
    m.Item = pyo.Set(initialize = list(pdata['items']), 
                     doc = 'list of items')
    # var
    m.x = pyo.Var(m.Item, domain = pyo.Binary, 
                  doc = '1 if item is part of knappsack')
    ## param
    @m.Param(m.Item, doc ='value of item i')
    def value(m,i):
        return pdata.loc[pdata['items'] == i,'values'].values[0]
    
    @m.Param(m.Item, doc ='weight of item i')
    def weight(m,i):
        return pdata.loc[pdata['items'] == i,'weights'].values[0]
    
    @m.Param(doc = 'capacity limit')
    def cap_limit(m):
        return list(pdata['capacity'].unique())[0]
    
    ## objective
    m.OBJ = pyo.Objective(expr = sum(m.value[i] * m.x[i] for i in m.Item), sense = pyo.maximize)
    
    ## constraints
    @m.Constraint(doc = 'knappsack capacity limit')
    def c1(m):
        return sum(m.weight[i] * m.x[i] for i in m.Item) <= m.cap_limit
    
    solver = pyo.SolverFactory(data['solver'])
    solver.solve(m)
    
    return m

In [ ]:
m = knappsack_01(data)

In [ ]:
def extract_solution(m):
    solution = {r: [pyo.value(m.x[r])==1.0] for r in m.Item}
    output = pd.DataFrame(data = solution)
    output.index = ['object in knappsack']
    return output

In [ ]:
extract_solution(m)

,a,b,c,d
object in knappsack,True,False,True,True
